<a href="https://colab.research.google.com/github/Geo6g/Classification/blob/main/georgy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [2]:
df=pd.read_csv('churn_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'churn_data.csv'

In [ ]:
print(df.info())


In [ ]:
print(df.head())

In [ ]:
print(df.isnull().sum())

In [ ]:
df = df.dropna()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = {}
for col in df.select_dtypes(include=['object']):
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoder[col] = le

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns.drop('churn')
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.countplot(x='churn', data=df)
plt.title('Churn Distribution')
plt.show()

In [ ]:
sns.heatmap(df.corr(), annot=True)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
sns.boxplot(x='churn', y='satisfaction', data=df)
plt.title('Satisfaction vs Churn')
plt.xlabel('Churn')
plt.ylabel('Satisfaction')
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
X=df.drop('churn',axis=1)
y=df['churn']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
models={
    'Logistic Regression':LogisticRegression(),
    'Random Forest':RandomForestClassifier(),
    'Decision Tree':DecisionTreeClassifier()
}

In [ ]:
for name,model in models.items():
    model.fit(X_train,y_train)
    print(f'{name} Accuracy:{model.score(X_test,y_test):.2f}')

In [ ]:
from sklearn.model_selection import GridSearchCV
rf_param_grid = {
    'n_estimators': [100],
    'max_depth': [None, 10],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'bootstrap': [True]
}

rf = RandomForestClassifier(random_state=42)
rf_grid_search = GridSearchCV(estimator=rf, param_grid=rf_param_grid,cv=5,verbose=1,n_jobs=-1)
rf_grid_search.fit(X_train, y_train)
print("Best Random Forest Parameters:", rf_grid_search.best_params_)

In [ ]:
lr_param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']  # Only solver supporting both l1 and l2
}

lr = LogisticRegression(max_iter=1000, random_state=42)

lr_grid_search = GridSearchCV(estimator=lr,
                              param_grid=lr_param_grid,
                              cv=5,
                              verbose=1,
                              n_jobs=-1)

lr_grid_search.fit(X_train, y_train)
print("Best Logistic Regression Parameters:", lr_grid_search.best_params_)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier

In [ ]:
best_rf = rf_grid_search.best_estimator_
y_pred_rf = best_rf.predict(X_test)

print(" Random Forest")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


In [ ]:
best_lr = lr_grid_search.best_estimator_
y_pred_lr = best_lr.predict(X_test)

print(" Logistic Regression")
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))


In [ ]:
dt_default = DecisionTreeClassifier(random_state=42)
dt_default.fit(X_train, y_train)
y_pred_dt = dt_default.predict(X_test)

print(" Decision Tree (Default)")
print("Accuracy:", accuracy_score(y_test, y_pred_dt))
print(classification_report(y_test, y_pred_dt))


In [ ]:
import matplotlib.pyplot as plt

models = ['Random Forest', 'Decision Tree', 'Logistic Regression']
accuracies = [0.9892, 0.9771, 0.7601]

plt.bar(models, accuracies, color=['green', 'blue', 'red'])
plt.ylim(0, 1)
plt.ylabel('Accuracy')
plt.title('Model Accuracy Comparison')
plt.show()


In [ ]:
acc_rf = rf_grid_search.best_estimator_.score(X_test, y_test)
acc_lr = lr_grid_search.best_estimator_.score(X_test, y_test)
acc_dt = DecisionTreeClassifier(random_state=42).fit(X_train, y_train).score(X_test, y_test)
accuracies = {
    'Random Forest': acc_rf,
    'Logistic Regression': acc_lr,
    'Decision Tree': acc_dt
}
best_model = max(accuracies, key=accuracies.get)
print(f"Best model is: {best_model} with accuracy: {accuracies[best_model]:.4f}")

In [ ]:

X_selected = X_test.iloc[[92, 93]]


y_pred = best_rf.predict(X_selected)
y_proba = best_rf.predict_proba(X_selected)[:, 1]


for i, idx in enumerate([92, 93]):
    print(f"\nSample {idx}")
    print(X_selected.iloc[i])
    print(" Prediction:", "Churn" if y_pred[i] == 1 else "No Churn")
    print(f" Probability of Churn: {y_proba[i]:.2f}")


 Random Forest was the most accurate model for predicting churn, outperforming
 Logistic Regression and Decision Tree.

 Low satisfaction level and longer tenure were strong indicators of higher employee churn.

 Encoded categorical features (department, salary) and normalized numerical data were used for model training.

Sample predictions showed reliable performance, with churn probabilities giving clear risk insights.

